In [20]:
import wptools, wikipedia

wptools.page('Google (company)').get_parse().data['infobox']

en.wikipedia.org (parse) Google (company)
en.wikipedia.org (imageinfo) File:Googleplex HQ (cropped).jpg
Google (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Googleple...
  infobox: <dict(23)> name, logo, logo_size, logo_alt, logo_captio...
  iwlinks: <list(8)> https://commons.wikimedia.org/wiki/Google, ht...
  pageid: 1092923
  parsetree: <str(231847)> <root><template><title>About</title><pa...
  requests: <list(2)> parse, imageinfo
  title: Google
  wikibase: Q95
  wikidata_url: https://www.wikidata.org/wiki/Q95
  wikitext: <str(195921)> {{About|the company|the search engine|Go...
}


{'name': 'Google, LLC',
 'logo': 'Google 2015 logo.svg',
 'logo_size': '250px',
 'logo_alt': 'Each letter of "Google" is colored (from left to right) in blue, red, yellow, blue, green, and red.',
 'logo_caption': 'Logo {{as of|2015|lc|=|y|since|=|y}}',
 'image': 'Googleplex HQ (cropped).jpg',
 'image_size': '250px',
 'image_caption': "Google's headquarters, the [[Googleplex]]",
 'former_name': 'Google Inc. (1998–2017)',
 'type': '[[Limited liability company]], [[Subsidiary]]',
 'industry': '{{Unbulleted list\n| [[Internet]]\n| [[Cloud computing]]\n| [[Computer software]]\n| [[Computer hardware]]\n| [[Artificial intelligence]]\n| [[Advertising]]}}',
 'founded': "{{Start date and age|1998|09|04}} {{efn|Google was originally incorporated on September 4, 1998, however, since 2002, the company has celebrated its anniversaries on various days in September, most frequently on September 27.|ref|{{cite web |url=https://time.com/3250807/google-anniversary/ |title=Google Used to Be the Company Th